In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv("../datasets/Movie Rating Prediction data/Train/Train.csv")

In [3]:
df.head(10)

,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos
5,Steve Carell comes into his own in his first s...,pos
6,I'm only going to write more because it's requ...,neg
7,"OK, it was a ""risky"" move to rent this flick, ...",neg
8,"Cannibalism, a pair of cinematic references to...",pos
9,This is one of the great modern kung fu films....,pos


In [4]:
df.isnull().sum()

review    0
label     0
dtype: int64

In [5]:
dfx = df.iloc[:,0].values
dfy = df.iloc[:,1].values

In [31]:
dfx[0]

"mature intelligent and highly charged melodrama unbelivebly filmed in China in 1948. wei wei's stunning performance as the catylast in a love triangle is simply stunning if you have the oppurunity to see this magnificent film take it"

In [6]:
print(df.shape)
print(dfx.shape)
print(dfy.shape)

(40000, 2)
(40000,)
(40000,)


In [7]:
temp = [x for x in dfy]
y_train = [1 if x == 'pos' else 0 for x in temp]

In [8]:
len(y_train)

40000

### Cleaning the training data

In [2]:
sw = stopwords.words('english')
sw.remove('not')
sw = set(sw)

ps = PorterStemmer()

In [3]:
print(sw)

{'because', 'same', 'and', 'whom', 'into', "shouldn't", 'by', 'yours', 'nor', 'as', 'those', 'when', 'once', 'what', 'more', "won't", 'having', 'other', 'herself', 'o', "couldn't", 'below', 'so', 'isn', "doesn't", 'ma', 'll', "you're", 'couldn', 'again', 'you', 'each', 'down', 'why', 'here', 'don', 'now', 'where', 'that', 'been', "weren't", 'does', 'its', 'any', "hadn't", 'ours', 'against', 'but', 'he', 'did', 'on', 'above', 'no', 'while', 'ain', 'won', 'who', 'there', 'hers', 'mightn', 'too', 'was', 'has', 'between', 'she', 'with', 'out', 'under', "you'll", 'are', 'ourselves', 'mustn', 'an', "should've", 'some', 's', "mightn't", 'both', 'my', 've', 't', 'aren', 'very', "it's", 'be', 'further', "mustn't", 'hadn', 're', 'have', 'myself', 'had', 'doesn', 'doing', 'if', 'm', 'wouldn', 'yourselves', 'of', "shan't", 'shan', 'until', 'shouldn', 'just', 'yourself', "you'd", 'they', 'most', 'such', 'd', 'needn', 'over', 'own', 'few', "wasn't", 'through', 'from', 'am', 'after', 'which', 'her', 

In [10]:
def cleaning_pipeline(review):
    words = word_tokenize(review.lower())
    words = [ps.stem(word) for word in words if word not in sw and word.isalpha()]
    review = " ".join(words)
    return review

In [11]:
cleaned_reviews = [ cleaning_pipeline(review) for review in dfx]

In [12]:
len(cleaned_reviews)

40000

### Loading and cleaning testing data


In [5]:
dftest = pd.read_csv('../datasets/Movie Rating Prediction data/Test/Test.csv').values

In [14]:
dftest.shape

(10000, 1)

In [15]:
test_reviews = dftest.reshape((-1,))

In [16]:
test_reviews.shape

(10000,)

In [17]:
cleaned_test_rev = [cleaning_pipeline(review) for review in test_reviews]

In [18]:
len(cleaned_test_rev)

10000

### Vectorization

In [19]:
cv = CountVectorizer(ngram_range=(1,3))
x_train_vect = cv.fit_transform(cleaned_reviews)

In [20]:
x_train_vect.shape

(40000, 6237829)

In [21]:
x_test_vect = cv.transform(cleaned_test_rev)

In [22]:
cv.vocabulary_

{'matur': 3380346,
 'intellig': 2759579,
 'highli': 2544059,
 'charg': 915111,
 'melodrama': 3418945,
 'unbelivebl': 5757838,
 'film': 1955955,
 'china': 948315,
 'wei': 6003254,
 'stun': 5303874,
 'perform': 4081692,
 'catylast': 855913,
 'love': 3222162,
 'triangl': 5685383,
 'simpli': 5003696,
 'oppurun': 3943442,
 'see': 4808231,
 'magnific': 3277109,
 'take': 5402432,
 'matur intellig': 3380614,
 'intellig highli': 2760352,
 'highli charg': 2544215,
 'charg melodrama': 915533,
 'melodrama unbelivebl': 3419457,
 'unbelivebl film': 5757839,
 'film china': 1964670,
 'china wei': 948821,
 'wei wei': 6003275,
 'wei stun': 6003271,
 'stun perform': 5304596,
 'perform catylast': 4083157,
 'catylast love': 855914,
 'love triangl': 3236001,
 'triangl simpli': 5685562,
 'simpli stun': 5006924,
 'stun oppurun': 5304565,
 'oppurun see': 3943443,
 'see magnific': 4819866,
 'magnific film': 3277345,
 'film take': 2010477,
 'matur intellig highli': 3380617,
 'intellig highli charg': 2760353,
 'h

### 1.Multinomial NB

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
mnb = MultinomialNB()

In [26]:
#Trainig
mnb.fit(x_train_vect,y_train)

MultinomialNB()

In [27]:
#Prediction
prediction = mnb.predict(x_test_vect)

In [28]:
type(prediction)

numpy.ndarray

In [29]:
#score
mnb.score(x_train_vect,y_train)

0.99975

In [30]:
#dumping the model
import pickle
with open("../artifacts/MNBmodel3.pkl", "wb") as f:
    pickle.dump((cv,mnb),f)